# CASO 1  - Tolerancia al riesgo

*Caso tomado de: Tatsat, H., Puri, S., & Lookabaugh, B. (2020). Machine Learning and Data Science Blueprints for Finance. O'Reilly media.*

**Detalle del caso**

<p style="text-align: justify;">
La empresa de asesoramiento financiero AGUERRE S.A. se especializa en la gestión de portafolios de inversión de diversos clientes. Para optimizar las inversiones, evalúa el perfil de riesgo de cada cliente en función de sus ingresos, objetivos de inversión (ahorro, mantenimiento de valor, entre otros) y otros factores que permiten definir un portafolio acorde a sus necesidades.
Actualmente, el análisis manual del perfil de riesgo consume demasiado tiempo y se ha observado que existen patrones recurrentes en los perfiles de los clientes, aunque también se dan casos excepcionales. Generalmente, para determinado tipo de clientes con un nivel de riesgo similar, los asesores de la empresa han cumplido con las finalidades y las expectativas de rentabilidad de los clientes, optimizando mediante el modelo de Markowitz.
</p>
    
<p style="text-align: justify;">
Por este motivo, AGUERRE S.A. te ha contratado para ayudar a automatizar este análisis y, si es posible, desarrollar una aplicación que facilite la evaluación del perfil de riesgo de los clientes, y además permita encontrar fácilmente un portafolio óptimo.
</p>

<p style="text-align: justify;">
La empresa cuenta con una base de datos de clientes con cararacterísticas sociodemográficas, ingresos, entre otros. Los asesores de la empresa, luego de un minucioso estudio de cada inversor, y en función de su conocimiento avanzado en Finanzas, han establecido un índice de riesgo del mismo.
Por otra parte, usted ha descargado datos de SP 500 de diversos activos, a efectos de poder formar un portafolio con los activos financieros con los que más comercia la empresa.
</p>

## 1) y 2) Entender el negocio y  problema

Es una empresa especializada en inversiones, que aconseja a sus clientes en qué deben invertir. La empresa debe seleccionar cuidadosamente el portafolio óptimo que satisfaga su finalidad, y obtener la mayor rentabilidad dado el riesgo que está dispuesto a asumir el cliente (a mayor rentabilidad, mayor el riesgo).

La empresa establece que quiere disminuir los tiempos de análisis del perfil de riesgo., por lo que parecería correcto que pretenda automatizarlo, siempre que ello sea posible y que dicha automatización sea confiable. Como la empresa ya tiene una base de datos de clientes con diversas características, y además cuenta con una variable referente a la tolerancia al riesgo por el cliente, y es determinada por expertos de la empresa, entonces es posible realizar modelos de machine learning supervisado para automatizar el análisis del perfil de riesgo de nuevos clientes. Un supuesto fuerte de hacer esto, es que damos por bueno el análisis que hicieron los asesores de la empresa. 

Posteriormente, si elaboramos un modelo que sea satisfactorio, use puede hacer un deploy del mismo, incorporándolo a un dashboard donde, ingresando las características del inversor, se pueda predecir el perfil de riesgo del mismo (el índice). Con este índice de tolerancia al riesgo, se puede automatizar la selección del portafolio óptimo para ese riesgo usando el modelo de Markowitz, lo cual creando así un robo-advisor.

## 3) Análisis del set de datos


In [11]:
# primero cargo las librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [134]:
#Cargo el set de datos
df = pd.read_csv('data_investors.csv',sep=";",decimal=",")

In [135]:
#veo los primeros 5 registros
df.head()

,YY1,Y1,WGT09,AGE07,AGECL07,EDUC07,EDCL07,MARRIED07,KIDS07,LIFECL07,...,NHMORTPCT,WAGEINCPCT,BUSSEFARMINCPCT,INTDIVINCPCT,KGINCPCT,SSRETINCPCT,TRANSFOTHINCPCT,PSAVINGPCT,LEVERAGEPCT,I
0,1,11,11668.13420,47,3,12,2,1,0,2,...,-21.052632,-32.931828,0.0,15939.27894,0.0,0.0,0.0,93.125197,270.403054,57
1,1,12,11823.45649,47,3,12,2,1,0,2,...,-21.052632,-32.931828,0.0,15939.27894,0.0,0.0,0.0,93.125197,249.593620,57
2,1,13,11913.22835,47,3,12,2,1,0,2,...,-50.000000,-32.931828,0.0,15939.27894,0.0,0.0,0.0,93.125197,209.233358,57
3,1,14,11929.39427,47,3,12,2,1,0,2,...,-33.333333,-32.931828,0.0,15939.27894,0.0,0.0,0.0,93.125197,209.273158,57
4,1,15,11917.72291,47,3,12,2,1,0,2,...,-38.596491,-32.931828,0.0,15939.27894,0.0,0.0,0.0,93.125197,232.690767,57


In [136]:
#tamaño
df.shape

(19285, 515)

In [138]:
#info general
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19285 entries, 0 to 19284
Columns: 515 entries, YY1 to I
dtypes: float64(212), int64(303)
memory usage: 75.8 MB


In [142]:
#me fijo estadísticos de  variables numéricas
df.describe()

,YY1,Y1,WGT09,AGE07,AGECL07,EDUC07,EDCL07,MARRIED07,KIDS07,LIFECL07,...,NHMORTPCT,WAGEINCPCT,BUSSEFARMINCPCT,INTDIVINCPCT,KGINCPCT,SSRETINCPCT,TRANSFOTHINCPCT,PSAVINGPCT,LEVERAGEPCT,I
count,19285.000000,19285.000000,19285.000000,19285.000000,19285.000000,19285.000000,19285.000000,19285.000000,19285.000000,19285.000000,...,1.928500e+04,1.928500e+04,1.928500e+04,1.928500e+04,1.928500e+04,1.928500e+04,1.928500e+04,1.928500e+04,1.928500e+04,19285.0
mean,2222.613689,22229.136894,5868.854779,51.647654,3.228157,14.072855,3.055535,1.314752,0.876588,3.792585,...,9.768241e+05,7.842138e+05,5.230099e+05,1.513447e+05,4.315485e+05,2.546299e+05,3.037898e+05,5.416303e+05,4.306108e+05,57.0
std,1275.293152,12752.931600,3881.525292,15.662512,1.494929,2.733539,1.055146,0.464429,1.200590,1.598757,...,1.142686e+07,1.869723e+07,1.360353e+07,5.184665e+06,3.384363e+07,4.677663e+06,3.916706e+06,2.036640e+07,2.116043e+07,0.0
min,1.000000,11.000000,0.885319,19.000000,1.000000,-1.000000,1.000000,1.000000,0.000000,1.000000,...,-1.000000e+02,-1.000000e+02,-1.016805e+08,-2.603585e+02,-9.962049e+07,-1.000000e+02,-8.168880e+06,-1.009656e+02,-1.000000e+02,57.0
25%,1130.000000,11302.000000,1992.217143,40.000000,2.000000,12.000000,2.000000,1.000000,0.000000,3.000000,...,-5.405405e+00,-2.115196e+01,-6.457549e+00,-2.867388e+01,0.000000e+00,0.000000e+00,0.000000e+00,-3.562493e+01,-2.584808e+01,57.0
50%,2233.000000,22333.000000,6756.245225,51.000000,3.000000,14.000000,3.000000,1.000000,0.000000,4.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.794657e+01,0.000000e+00,57.0
75%,3323.000000,33234.000000,8538.990444,62.000000,4.000000,16.000000,4.000000,2.000000,2.000000,5.000000,...,0.000000e+00,1.692806e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.225191e+02,6.344615e+01,57.0
max,4423.000000,44235.000000,17895.936630,95.000000,6.000000,17.000000,4.000000,2.000000,10.000000,6.000000,...,5.000000e+08,1.095825e+09,7.471537e+08,3.094213e+08,1.870849e+09,2.717647e+08,1.225332e+08,8.860001e+08,1.721023e+09,57.0


In [143]:
#me fijo si hay missings
nan_counts = df.isna().sum()
nan_counts

YY1                0
Y1                 0
WGT09              0
AGE07              0
AGECL07            0
                  ..
SSRETINCPCT        0
TRANSFOTHINCPCT    0
PSAVINGPCT         0
LEVERAGEPCT        0
I                  0
Length: 515, dtype: int64

In [146]:
#me fijo cuántos tienen nan
nan_counts[nan_counts > 0]

Series([], dtype: int64)

In [148]:
#tipo de variables
df.dtypes.value_counts()

int64      303
float64    212
Name: count, dtype: int64

In [ ]:
#chequeo duplicados